In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
url = "https://www.cuny.edu/academics/academic-calendars/fall-2021/"
response = requests.get(url)